# Lib Imports

In [1]:
import vtk
import pandas as pd
import numpy as np
from vtk.util.numpy_support import numpy_to_vtk

import vtkmodules.vtkInteractionStyle
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderer
)


# Initial Variables Definitions and Data Read

In [2]:
#Define Colors
colors = vtkNamedColors()

# Read structures data
buildings = pd.read_csv('../../Structures/buildings.txt', sep='\t')
trees = pd.read_csv('../../Structures/trees_baseline.txt', sep='\t')
roads = pd.read_csv('../../Structures/roads.txt', sep='\t')


# Load the NOx data
nox_data = pd.read_csv('../../NOx/B_NOx_12.dat', sep='\s+')

#convert numbers as strings to floats 
nox_data['vel'] = pd.to_numeric(nox_data['vel'], errors='coerce')
nox_data['dir'] = pd.to_numeric(nox_data['dir'], errors='coerce')

# Convert the direction from degrees to radians
nox_data['dir_rad'] = np.radians(nox_data['dir'])



# Functions Definitions for the structures

In [3]:
#Function Create Building

def create_building(x, y, z, id):
    # Define the eight vertices of the cube
    points = vtk.vtkPoints()
    points.InsertNextPoint(x[0], y[0], z)  # 0: bottom-left-front
    points.InsertNextPoint(x[1], y[1], z)  # 1: bottom-right-front
    points.InsertNextPoint(x[2], y[2], z)  # 2: top-right-front
    points.InsertNextPoint(x[3], y[3], z)  # 3: top-left-front
    points.InsertNextPoint(x[0], y[0], 0)  # 4: bottom-left-back
    points.InsertNextPoint(x[1], y[1], 0)  # 5: bottom-right-back
    points.InsertNextPoint(x[2], y[2], 0)  # 6: top-right-back
    points.InsertNextPoint(x[3], y[3], 0)  # 7: top-left-back

    # Define the six faces of the cube (topology)
    faces = vtk.vtkCellArray()

    face = vtk.vtkPolygon()
    face.GetPointIds().SetNumberOfIds(4)
    for j in range(4):  # bottom face
        face.GetPointIds().SetId(j, j + 4)
    faces.InsertNextCell(face)

    face = vtk.vtkPolygon()
    face.GetPointIds().SetNumberOfIds(4)
    for j in range(4):  # top face
        face.GetPointIds().SetId(j, j)
    faces.InsertNextCell(face)

    for i in range(4):  # side faces
        face = vtk.vtkPolygon()
        face.GetPointIds().SetNumberOfIds(4)
        face.GetPointIds().SetId(0, i)
        face.GetPointIds().SetId(1, (i + 1) % 4)
        face.GetPointIds().SetId(2, (i + 1) % 4 + 4)
        face.GetPointIds().SetId(3, i + 4)
        faces.InsertNextCell(face)

    # Create a polydata object
    cube = vtk.vtkPolyData()

    # Set the points and polygons
    cube.SetPoints(points)
    cube.SetPolys(faces)

    # Create mapper and actor
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(cube)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(colors.GetColor3d('Yellow'))


    return actor


In [4]:
#Function Create Tree

def create_tree(x, y, z, id):
    # Define the eight vertices of the cube
    points = vtk.vtkPoints()
    points.InsertNextPoint(x[0], y[0], z)  # 0: bottom-left-front
    points.InsertNextPoint(x[1], y[1], z)  # 1: bottom-right-front
    points.InsertNextPoint(x[2], y[2], z)  # 2: top-right-front
    points.InsertNextPoint(x[3], y[3], z)  # 3: top-left-front
    points.InsertNextPoint(x[0], y[0], 0)  # 4: bottom-left-back
    points.InsertNextPoint(x[1], y[1], 0)  # 5: bottom-right-back
    points.InsertNextPoint(x[2], y[2], 0)  # 6: top-right-back
    points.InsertNextPoint(x[3], y[3], 0)  # 7: top-left-back

    # Define the six faces of the cube (topology)
    faces = vtk.vtkCellArray()

    face = vtk.vtkPolygon()
    face.GetPointIds().SetNumberOfIds(4)
    for j in range(4):  # bottom face
        face.GetPointIds().SetId(j, j + 4)
    faces.InsertNextCell(face)

    face = vtk.vtkPolygon()
    face.GetPointIds().SetNumberOfIds(4)
    for j in range(4):  # top face
        face.GetPointIds().SetId(j, j)
    faces.InsertNextCell(face)

    for i in range(4):  # side faces
        face = vtk.vtkPolygon()
        face.GetPointIds().SetNumberOfIds(4)
        face.GetPointIds().SetId(0, i)
        face.GetPointIds().SetId(1, (i + 1) % 4)
        face.GetPointIds().SetId(2, (i + 1) % 4 + 4)
        face.GetPointIds().SetId(3, i + 4)
        faces.InsertNextCell(face)

    # Create a polydata object
    cube = vtk.vtkPolyData()

    # Set the points and polygons
    cube.SetPoints(points)
    cube.SetPolys(faces)

    # Create a mapper and actor
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(cube)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(colors.GetColor3d('Green'))


    return actor


In [5]:
#Function Create Roads

def create_road(x, y, id):
    # Define the vertices of the road
    points = vtk.vtkPoints()
    points.InsertNextPoint(x[0], y[0], 0)  # 0: bottom-left-front
    points.InsertNextPoint(x[1], y[1], 0)  # 1: bottom-right-front
    points.InsertNextPoint(x[2], y[2], 0)  # 2: bottom-left-back
    points.InsertNextPoint(x[3], y[3], 0)  # 3: bottom-right-back
    

    # Define the face of the road (topology)
    faces = vtk.vtkCellArray()

    face = vtk.vtkPolygon()
    face.GetPointIds().SetNumberOfIds(4)
    for j in range(4):
        face.GetPointIds().SetId(j, j + 1)
    faces.InsertNextCell(face)


    # Create a polydata object
    rectangle = vtk.vtkPolyData()

    # Set the points and polygons
    rectangle.SetPoints(points)
    rectangle.SetPolys(faces)

    # Create a mapper and actor
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(rectangle)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(colors.GetColor3d('DarkGrey'))

    return actor

# Pollution values calculations

In [6]:

# calculating the vectors for velocity
velocity_vectors = np.vstack([nox_data['vel']*np.sin(nox_data['dir']), nox_data['vel']*np.cos(nox_data['dir']), np.zeros_like(nox_data['vel'])]).T

# Create the vtkPoints object and add the coordinates to it
points = vtk.vtkPoints()
for i in range(len(nox_data)):
    points.InsertPoint(i, nox_data['x'][i], nox_data['y'][i], 0)

# Create the vtkPolyData object and add the points to it
nox_polydata = vtk.vtkPolyData()
nox_polydata.SetPoints(points)

# Convert the velocity and direction into a vector
vecs = vtk.vtkDoubleArray()
vecs.SetNumberOfComponents(3)
vecs.SetName("Vectors")


for i in range(len(nox_data)):
    # Insert points
    points.InsertPoint(i, nox_data['x'][i], nox_data['y'][i], 0)

    # Calculate vector components
    x_comp = nox_data['vel'][i] * np.cos(nox_data['dir_rad'][i])
    y_comp = nox_data['vel'][i] * np.sin(nox_data['dir_rad'][i])
    z_comp = 0

    # Insert vectors
    vecs.InsertTuple3(i, x_comp, y_comp, z_comp)

# Create the vtkPolyData object and add the points and vectors to it
nox_polydata.SetPoints(points)
nox_polydata.GetPointData().SetVectors(vecs)


# Convert the concentration into a scalar
concentration = numpy_to_vtk(nox_data['conc'].values)

# Add the concentration to the polydata
nox_polydata.GetPointData().SetScalars(concentration)

1

# Lookup Tables for Reversion of Color Gradient

#### Reversion of Default Table

In [7]:
# Define lookup table
lookupTable = vtk.vtkLookupTable()
lookupTable.SetNumberOfTableValues(256) # The number can be altered for a smoother gradient
lookupTable.Build()

# Reverse the lookup table
for i in range(256):
    lookupTable.SetTableValue(i, *lookupTable.GetTableValue(255-i))



#### Visualisation with only two colors

In [8]:
ctf = vtk.vtkColorTransferFunction()
ctf.AddRGBPoint(0, 0, 0, 0)  
ctf.AddRGBPoint(1, 1, 1, 0)
ctf.AddRGBPoint(2, 1, 0, 0)
#The first parameter decides which of the two will represent the color for the highest density region meanwhile the other 3 decide on the color chosen
#The higher the id (first parameter) of the colors, the higher the concentration it will be represention, with the highest accounting for the densest region
#There can be as many colors as the users want as long they are defined in new lines by incrementing the value of the first parameter

#Color Codes: 
# (x, 0, 0, 1) - blue
# (x, 0, 1, 0) - green
# (x, 1, 1, 0) - yellow
# (x, 1, 0, 0) - red
# (x, 1, 1, 1) - white
# (x, 0, 0, 0) - black


2

# Renderer Initialization

In [9]:
# Initialize the renderer
renderer = vtk.vtkRenderer()
#renderer.SetBackground(colors.GetColor3d('LightBlue')) #Background color makes the visualization more unclear


# Calling the functions for the structures

In [10]:

# For each building id, create the corresponding actor and add it to the renderer
for id in buildings['id'].unique():
    building_data = buildings[buildings['id'] == id]
    x = building_data['x'].values
    y = building_data['y'].values
    z = building_data['z'].values[0]  # height is the same for all points of a building
    actor = create_building(x, y, z, id)
    renderer.AddActor(actor)

    # For each tree id, create the corresponding actor and add it to the renderer
for id in trees['id'].unique():
    tree_data = trees[trees['id'] == id]
    x = tree_data['x'].values
    y = tree_data['y'].values
    z = tree_data['z'].values[0]  # height is the same for all points of a building
    actor = create_tree(x, y, z, id)
    renderer.AddActor(actor)

    # For each road id, create the corresponding actor and add it to the renderer
for id in roads['id'].unique():
    road_data = roads[roads['id'] == id]
    x = road_data['x'].values
    y = road_data['y'].values
    actor = create_road(x, y, id)
    renderer.AddActor(actor)


# Glyph Definitions and Pollution Mapper

In [11]:
# Create a cone
cone = vtk.vtkConeSource()

# Create a glyph3D object and set the cone as the source
glyph = vtk.vtkGlyph3D()
glyph.SetSourceConnection(cone.GetOutputPort())
glyph.SetInputData(nox_polydata)
glyph.SetVectorModeToUseVector()
glyph.SetScaleModeToScaleByVector()
glyph.SetColorModeToColorByScalar()
glyph.Update()

# Create a mapper and actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(glyph.GetOutputPort())

# COLOR REPRESENTATION FUNCTION CALL 
mapper.SetLookupTable(ctf)  #Comment this line to use default gradient, change the name inside the brackets to suit the color function you want 

actorp = vtk.vtkActor()
actorp.SetMapper(mapper)

renderer.AddActor(actorp)


# Create Render Window

In [12]:

# Create a render window and add the renderer to it
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.SetSize(900, 600)
render_window.SetWindowName('25 Abril Maquete com poluicao registada às 00:00')

# Create Interactor

In [13]:
# Create a render window interactor and set the render window for it
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)


# Start the visualization
interactor.Initialize()
interactor.Start()